In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from surprise import SVD,Dataset, Reader
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split

In [2]:
traindata = pd.read_csv('archive/interactions_train.csv')
traindata.head()

,user_id,recipe_id,date,rating,u,i
0,2046,4684,2000-02-25,5.0,22095,44367
1,2046,517,2000-02-25,5.0,22095,87844
2,1773,7435,2000-03-13,5.0,24732,138181
3,1773,278,2000-03-13,4.0,24732,93054
4,2046,3431,2000-04-07,5.0,22095,101723


In [4]:
reader = Reader(rating_scale=(traindata.rating.min(), traindata.rating.max()))
data = Dataset.load_from_df(traindata[['user_id', 'recipe_id', 'rating']], reader)

In [10]:
svd = SVD(n_epochs=10)
results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9165  0.9182  0.9171  0.9044  0.9167  0.9114  0.9210  0.9100  0.9214  0.9151  0.9152  0.0049  
MAE (testset)     0.5433  0.5456  0.5440  0.5405  0.5444  0.5401  0.5478  0.5414  0.5471  0.5441  0.5438  0.0025  
Fit time          6.28    6.36    6.62    6.92    6.86    6.78    6.67    6.70    6.42    6.72    6.63    0.21    
Test time         0.50    0.84    0.82    0.59    0.61    0.60    0.81    0.89    0.82    0.52    0.70    0.14    


In [13]:
print("Average MAE: ", np.average(results["test_mae"]))
print("Average RMSE: ", np.average(results["test_rmse"]))

Average MAE:  0.543840710845295
Average RMSE:  0.9151750914312521


In [14]:
param_grid = {
  'n_factors': [20, 50, 100],
  'n_epochs': [5, 10, 20]
}
 
gridSearch = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=10)
gridSearch.fit(data)
 
print(gridSearch.best_score['rmse'])
print(gridSearch.best_params['rmse'])

0.9116394839131333
{'n_factors': 20, 'n_epochs': 10}


In [29]:
best_factor = gridSearch.best_params['rmse']['n_factors']
best_epoch = gridSearch.best_params['rmse']['n_epochs']
 
 
svd = SVD(n_factors=best_factor, n_epochs=best_epoch)
trainset = data.build_full_trainset()
svd.fit(trainset)

In [30]:
testdata = pd.read_csv('archive/interactions_test.csv')
testdata.head()

,user_id,recipe_id,date,rating,u,i
0,8937,44551,2005-12-23,4.0,2,173538
1,56680,126118,2006-10-07,4.0,16,177847
2,349752,219596,2008-04-12,0.0,26,89896
3,628951,82783,2007-11-13,2.0,45,172637
4,92816,435013,2013-07-31,3.0,52,177935


In [31]:
svd.predict('56680','126118')

Prediction(uid='56680', iid='126118', r_ui=None, est=4.574089892559891, details={'was_impossible': False})

In [33]:
default_predict('56680','126118')

NameError: name 'default_predict' is not defined